In [2]:
import tensorcircuit as tc
import tensorflow as ts 
import numpy as np
import math

def i_iterate_multiplication(n):
    tmp = 1
    for i in range(n):
        tmp = np.kron(I, tmp)
    return tmp

n = 10
h_i = 1
H = 0
theta = math.pi/2
I = tc.gates._i_matrix
X = tc.gates._x_matrix  
Y = tc.gates._y_matrix  
Z = tc.gates._z_matrix 

for i in range(0, n-1):
    H = H + np.kron(np.kron(i_iterate_mutiplication(i), np.kron(Z, Z)), i_iterate_multiplication(n-2-i))
for i in range(0, n):
    H = H + h_i * (np.kron(np.kron(i_iterate_mutiplication(i), X), i_iterate_multiplication(n-1-i)))
print(H)

NameError: name 'i_iterate_mutiplication' is not defined

In [67]:
list_min_w 

[-4.0,
 -4.017524005494917,
 -4.0705348332685265,
 -4.1608741942551974,
 -4.2922565389467735,
 -4.4694903440054405,
 -4.696054456230239,
 -4.9713281182400815,
 -5.290050539692084,
 -5.644434276301777,
 -6.026674183332291,
 -6.430255873281567,
 -6.850187770158756,
 -7.282770024115734,
 -7.72527701883572,
 -8.175686878183676,
 -8.632480719393477,
 -9.094500755819269,
 -9.560851330919801,
 -10.03082972400129,
 -10.50387729773436,
 -10.979544559529472,
 -11.457465821886618,
 -11.937340564769775,
 -12.418919540221443,
 -12.901994278585947,
 -13.386389067550667,
 -13.87195475202254,
 -14.35856389114229,
 -14.846106938460796,
 -15.334489201794272,
 -15.823628403206218,
 -16.31345270526729,
 -16.803899102785568,
 -17.294912103362254,
 -17.78644263796001,
 -18.278447155980036,
 -18.770886869353365,
 -19.263727117749003,
 -19.756936832826028]

In [ ]:
print(list_min_h)


In [71]:
lmw = np.loadtxt('a.txt')
lmw

array([ -4.        ,  -4.01752401,  -4.07053483,  -4.16087419,
        -4.29225654,  -4.46949034,  -4.69605446,  -4.97132812,
        -5.29005054,  -5.64443428,  -6.02667418,  -6.43025587,
        -6.85018777,  -7.28277002,  -7.72527702,  -8.17568688,
        -8.63248072,  -9.09450076,  -9.56085133, -10.03082972,
       -10.5038773 , -10.97954456, -11.45746582, -11.93734056,
       -12.41891954, -12.90199428, -13.38638907, -13.87195475,
       -14.35856389, -14.84610694, -15.3344892 , -15.8236284 ,
       -16.31345271, -16.8038991 , -17.2949121 , -17.78644264,
       -18.27844716, -18.77088687, -19.26372712, -19.75693683])

In [74]:
import time
t_0=time.time()
from numpy import linalg as LA
from functools import partial
import numpy as np
import tensorflow as tf
#import jax
#from jax.config import config

#config.update("jax_enable_x64", True)
#from jax import numpy as jnp
#from jax.experimental import optimizers
import tensorcircuit as tc
import math
import matplotlib.pyplot as plt

t_1=time.time()
print(t_1-t_0,"s")

tc.set_backend("tensorflow")
K = tc.set_backend("tensorflow")
zz = np.kron(tc.gates._z_matrix, tc.gates._z_matrix)

qubit = 2
level = 2
batch = 1
h_round = 0
P = [ ]
list_min_w = [ ]
list_min_h = [ ]
lossss = [ ]

def ideal_min_h(n, h_range):

    def i_iterate_multiplication(n):
        tmp = 1
        for i in range(n):
            tmp = np.kron(I, tmp)
        return tmp

    h_i = 0
    H = 0
    theta = math.pi/2
    min_w = 999
    I = tc.gates._i_matrix
    X = tc.gates._x_matrix  
    Y = tc.gates._y_matrix  
    Z = tc.gates._z_matrix 
    history = [ ]
    f= open("myfile.txt","w+")
    f.write("n = %s\n" % (n))
    for p in range(h_range * 10):

        for i in range(0, n-1):
            H = H - np.kron(np.kron(i_iterate_multiplication(i), np.kron(Z, Z)), i_iterate_multiplication(n-2-i))
        for i in range(0, n):
            H = H + h_i * (np.kron(np.kron(i_iterate_multiplication(i), X), i_iterate_multiplication(n-1-i)))
        # print(H)

        w, v = LA.eig(H)

        for i in range(0, len(w)):
            if w[i] < min_w:
                min_w = w[i]
        print(h_i)
        print(min_w)
        f.write("h_i = %f      min_w = %f \n" %(h_i, min_w))
        h_i = h_i + 0.1
        list_min_w.append(min_w)
        min_w = 999
        H = 0


#     plt.plot([0.1*p for p in range(h_range * 10)], history)
#     plt.ylabel("value")
#     plt.xlabel("training step")

    f.close()

def min(h,qubit,h_round):
    global P
    def rzz(c,i,j, params):
        c.cnot(i,i+1)
        c.rz(i+1,theta=params[4*j+1, i])
        c.cnot(i,i+1)
        return c
    
    def find_min(np_array):
        tmp = 9999
        for m in range(len(np_array)):
            if np_array[m] < tmp:
                tmp = np_array[m]
        return tmp
    
    def energy(c: tc.Circuit):
        e = 0.0
        n = c._nqubits

        for i in range(n):
            e += h * c.expectation((tc.gates.x(), [i]))  # <X_i>
        for i in range(n - 1):  # OBC
            e += (-1.0) * c.expectation(
                (tc.gates.z(), [i]), (tc.gates.z(), [(i + 1) % n])
            )  # <Z_iZ_{i+1}>
        return tc.backend.real(e)

    def ex(params):
        c=tc.Circuit(qubit)
        k=level
        for i in range(qubit):
            c.h(i)
        for j in range(k):
            for i in range(qubit):
                c.ry(i,theta=params[4*j+3,i])
                c.rz(i,theta=params[4*j+2,i])
                c.s(i)
            for i in range(qubit):
                c.rx(i,theta=params[4*j, i])
            for i in range(qubit-1):
                c = rzz(c,i,j, params)
        return energy(c)
    def clean_doc():
        open('myfile.txt', 'w').close()
        
        
        
        
    ex_vg = tc.backend.jit(tc.backend.vvag(ex, argnums=0, vectorized_argnums=0))
    history = np.zeros([int(500*((1+h)**2)) + 1])
    opt = K.optimizer(tf.keras.optimizers.Adam(0.02))
    params = [ ]
#     if h_round == 0:
    params = tf.Variable(initial_value=tf.random.normal(shape=[batch, level * 4+3, qubit], stddev=0.1, dtype=getattr(tf, tc.rdtypestr)))
    for _ in range (int(100*((1+h)**2))):
        v, g = ex_vg(params)
        params = opt.update(g, params)

        history[_]=np.min(v.numpy())
#     else:
#         params = P
#         for _ in range(100):
#             v, g = ex_vg(params)
#             params = opt.update(g, params)
    #     if _ % 20 == 0:
    #         t_2=time.time()
    #         print(v)
    #         print(t_2-t_1,"s")
    #         t_1=t_2
    
#             history[_]=np.min(v.numpy())
    P = params

#     plt.plot([i for i in range(1000)], history)
#     plt.ylabel("value")
#     plt.xlabel("training step")
    # print((t_2-t_0)/60,"m")
    return find_min(history)

                
min_h = [ ]
ideal_min_h(2, 4)
f = open("myfile.txt", "a+")
f.write("\nCalculated Values:")
f.close()
for p in range(40):
    h = 0.1 * p
    min_h.append(min(h, qubit, h_round))
#     h_round = h_round + 1
    f = open("myfile.txt", "a+")
    f.write("\nh_i = %f      min_h = %f      loss = %f" % 
            (h, min_h[-1], min_h[-1] - list_min_w[p]))
    f.close()
    lossss.append(min_h[-1] - list_min_w[p])
    list_min_h.append(min_h[-1])
#     print("%s: %s" % (h, min_h[-1]))
# print(min_h)
h_round = 0
plt.plot([i*0.1 for i in range(h_range * 10)], lossss)
plt.ylabel("loss")
plt.xlabel("h")


0.0002512931823730469 s
0
-1.0
0.1
-1.0198039027185566
0.2
-1.0770329614269007
0.30000000000000004
-1.1661903789690602
0.4
-1.2806248474865696
0.5
-1.414213562373095
0.6
-1.5620499351813306
0.7
-1.7204650534085257
0.7999999999999999
-1.8867962264113212
0.8999999999999999
-2.059126028197398
0.9999999999999999
-2.23606797749979
1.0999999999999999
-2.4166091947189132
1.2
-2.5999999999999996
1.3
-2.7856776554368246
1.4000000000000001
-2.9732137494637016
1.5000000000000002
-3.1622776601683795
1.6000000000000003
-3.3526109228480423
1.7000000000000004
-3.544009029333871
1.8000000000000005
-3.736308338453882
1.9000000000000006
-3.929376540877702
2.0000000000000004
-4.1231056256176615
2.1000000000000005
-4.317406628984582
2.2000000000000006
-4.512205669071394
2.3000000000000007
-4.70744091837593
2.400000000000001
-4.9030602688525065
2.500000000000001
-5.099019513592786
2.600000000000001
-5.295280917949493
2.700000000000001
-5.491812087098396
2.800000000000001
-5.68858506133116
2.900000000000001

NameError: name 'h_range' is not defined

In [ ]:
np.savetxt('aa.txt',list_min_w, )
np.savetxt("bb.txt",list_min_h, )

In [ ]:
clean_doc()

In [75]:
print(list_min_h)


[-0.9999868273735046, -1.0108476877212524, -1.0449986457824707, -1.166190505027771, -1.2806251049041748, -1.4142143726348877, -1.5620503425598145, -1.7204656600952148, -1.8867965936660767, -2.05912709236145, -2.2360689640045166, -2.4166100025177, -2.600001335144043, -2.7856783866882324, -2.973215103149414, -3.1622791290283203, -3.3526132106781006, -3.544011116027832, -3.7363100051879883, -3.9293785095214844, -4.123106956481934, -4.317409038543701, -4.512207508087158, -4.707443714141846, -4.9030632972717285, -5.099022388458252, -5.295283794403076, -5.49181604385376, -5.688588619232178, -5.8855791091918945, -6.082766532897949, -6.280130863189697, -6.477658271789551, -6.6753315925598145, -6.87313985824585, -7.071070671081543, -7.269116401672363, -7.46726655960083, -7.665511608123779, -7.863846778869629]


In [78]:
list_min_w

[-1.0,
 -1.0198039027185566,
 -1.0770329614269007,
 -1.1661903789690602,
 -1.2806248474865696,
 -1.414213562373095,
 -1.5620499351813306,
 -1.7204650534085257,
 -1.8867962264113212,
 -2.059126028197398,
 -2.23606797749979,
 -2.4166091947189132,
 -2.5999999999999996,
 -2.7856776554368246,
 -2.9732137494637016,
 -3.1622776601683795,
 -3.3526109228480423,
 -3.544009029333871,
 -3.736308338453882,
 -3.929376540877702,
 -4.1231056256176615,
 -4.317406628984582,
 -4.512205669071394,
 -4.70744091837593,
 -4.9030602688525065,
 -5.099019513592786,
 -5.295280917949493,
 -5.491812087098396,
 -5.68858506133116,
 -5.885575587824868,
 -6.082762530298225,
 -6.280127387243035,
 -6.4776538962806605,
 -6.675327707311459,
 -6.873136110975835,
 -7.071067811865478,
 -7.2691127381545,
 -7.46726188103779,
 -7.665507158694724,
 -7.863841300535002]